<a href="https://colab.research.google.com/github/ChinaYiqun/DrugAI/blob/master/%E5%9B%BE%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E5%88%86%E5%AD%90%E9%A2%84%E6%B5%8B%E6%95%99%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装库

In [ ]:
!pip install deepchem
!curl -Lo rdkit_installer.py https://git.io/fxiPZ
import rdkit_installer
%time rdkit_installer.install()
# deepchem 库为分子、图、蛋白质领域深度学习框架 为tesorflow的二次封装 
# 文档为 https://deepchem.readthedocs.io/en/latest/index.html 文档中也包含了这个教程
# rdkit库 为分子预处理工具包，其中内置了很多分子预处理的工具，对分子预测很有帮助

# 导入数据

In [2]:
import deepchem as dc
import numpy as np
smiles = [
  'O=Cc1ccc(O)c(OC)c1',
  'CN1CCC[C@H]1c2cccnc2',
  'C1CCCCC1',
  'c1ccccc1',
  'CC(=O)O',
]
# 五个分子式

# 提取特征

In [3]:
properties = [0.4, -1.5, 3.2, -0.2, 1.7]
# 这个是标签 随机给的

featurizer = dc.feat.CircularFingerprint(size=1024)
ecfp = featurizer.featurize(smiles)
ecfp.shape
# 
# 其它的特征提取方法 (包含图神经网络向量化) 参考： https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html?highlight=featurizer
# 可以直接调用的图神经网络向量化方法为 
  # ConvMolFeaturizer
  # WeaveFeaturizer
  # MolGanFeaturizer
  # MolGraphConvFeaturizer
# 目前最好方法为：Self-Supervised Graph Transformer on Large-Scale Molecular Data https://github.com/tencent-ailab/grover deep chem 没有收录在deepchem中
# 这几种方法都是图网络的方法，每个方法都对应着论文，可以知乎搜一下了解学习


(5, 1024)

# 数据集划分

In [4]:
dataset = dc.data.NumpyDataset(X=ecfp, y=np.array(properties))
splitter = dc.splits.RandomSplitter()
# split 5 datapoints in the ratio of train:valid:test = 3:1:1
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(
  dataset=dataset, frac_train=0.6, frac_valid=0.2, frac_test=0.2
)

# 训练与测试

In [5]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
model = dc.models.SklearnModel(model=rf)
# model training
model.fit(train_dataset)
valid_preds = model.predict(valid_dataset)

In [6]:
valid_preds

array([1.926])

接下来的实验：
可以换自己的数据进行实验，下载drugcentral网站 https://drugcentral.org/ 
有关高血压疗效的数据集，进行训练预测。 
例 ：https://drugcentral.org/?q=high+blood+pressure